This is a sample for celestial navigation for a stationary observer<br> 
© August Linnman, 2025, email: august@linnman.net<br>
MIT License (see [LICENSE file](LICENSE))

Jupyter notebook for ordinary three-fix celestial navigation (stationary).

Run it once to create the input form. Used data will be saved for later sessions. 


In [103]:
# Importing Python libraries
from time import time
from starfix import Sight, SightCollection, get_representation, get_google_map_string,\
                    IntersectError, LatLonGeodetic

In [ ]:
# Input form
import ipywidgets as widgets
from ipywidgets import VBox, Layout
import json

file_name = "notebook1-dict.3.json"

def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

try:
    with open(file_name, "r") as f:
        s = f.read ()
        numDict = json.loads (s)
except FileNotFoundError:
    numDict = {"ObjectName1" : "Sun",
               "Altitude1" : "55:7:54.4",
               "Time1" : "2024-05-05 15:55:18+00:00",
               "LimbCorrection1" : "0",
               "IndexError1" : "0",
               "ArtificialHorizon1" : "False",
               "ObserverHeight1" : "0",
               "Temperature1" : "10",
               "TemperatureGradient1" : "-0.01",
               "Pressure1" : "101",
               
               "ObjectName2" : "Sun",
               "Altitude2" : "19:28:27.4",
               "Time2" : "2024-05-05 23:01:19+00:00",
               "LimbCorrection2" : "0",
               "IndexError2" : "0",
               "ArtificialHorizon2" : "False",
               "ObserverHeight2" : "0",
               "Temperature2" : "10",
               "TemperatureGradient2" : "-0.01",
               "Pressure2" : "101",                                             
               
               "ObjectName3" : "Vega",
               "Altitude3" : "30:16:15.7",
               "Time3" : "2024-05-06 04:04:13+00:00",
               "LimbCorrection3" : "0",
               "IndexError3" : "0",
               "ArtificialHorizon3" : "False",
               "ObserverHeight3" : "0",               
               "Temperature3" : "10",
               "TemperatureGradient3" : "-0.01",                
               "Pressure3" : "101",               

               "DrpLat" : "40",
               "DrpLon" : "-90",
               
               "Use1" : "True",
               "Use2" : "True",
               "Use3" : "True"}

def dump_dict ():
    jDump = json.dumps (numDict)
    with open(file_name, "w") as f:
        f.write(jDump)    

def handle_change (change):
    if change['type'] == 'change' and change['name'] == 'value':
        the_owner = change['owner']
        # assert isinstance (the_owner, MyWidget)
        the_owner.handle_event (change)

style = {'description_width': '120px'}
class MyTextWidget (widgets.Text):

    def __init__ (self, attrName, description):
        self.__attrName = attrName
        super().__init__ (numDict[self.__attrName], description=description, disabled=False,
                          style=style,
                          layout=Layout(width='50%'))
        self.observe (handle_change)

    def handle_event (self, change):
        numDict[self.__attrName]=change['new']
        dump_dict ()

class MyCheckboxWidget (widgets.Checkbox):

    def __init__ (self, attrName, description):
        self.__attrName = attrName
        super().__init__ (str2bool(numDict[self.__attrName]), description=description, disabled=False,
                          style=style)
                         
        self.observe (handle_change)

    def handle_event (self, change):
        numDict[self.__attrName]=str(change['new'])
        dump_dict ()

class MyLimbDropdown (widgets.Dropdown):
    def __init__ (self, attrName, description):
        self.__attrName = attrName
        super().__init__ (value=numDict[self.__attrName],
                          options=[("Lower",'-1'),("Central", '0'),("Upper",'1')],
                          description=description, disabled=False, style=style)
        self.observe (handle_change)

    def handle_event (self, change):
        numDict[self.__attrName]=str(change['new'])
        dump_dict ()            

widget_array = []
typeArray = [["ObjectName",          "NAME",           MyTextWidget],
             ["Altitude",            "ALT",            MyTextWidget],
             ["Time",                "TIME",           MyTextWidget],
             ["IndexError",          "INDEX_ERROR",    MyTextWidget],
             ["LimbCorrection",      "LIMB_CORR",      MyLimbDropdown],
             ["ArtificialHorizon",   "ARTIFICIAL_HOR", MyCheckboxWidget],
             ["ObserverHeight",      "OBS_HEIGHT",     MyTextWidget],
             ["Temperature",         "TEMP",           MyTextWidget],
             ["TemperatureGradient", "TEMP_GRADIENT",  MyTextWidget],
             ["Pressure",            "PRESSURE",       MyTextWidget]]

widget_array.append (MyTextWidget ("DrpLat","DRP_LAT"))
widget_array.append (MyTextWidget ("DrpLon","DRP_LON"))
for i in range (3):
    widget_array.append (MyCheckboxWidget("Use"+str(i+1),\
                                          description="Use " + str(i+1)))
    for j in range (len(typeArray)):
        cl = typeArray[j][2]
        assert isinstance (cl, type)
        obj = cl (typeArray[j][0]+str(i+1), description=typeArray[j][1]+"_"+str(i+1))
        widget_array.append (obj)

VBox (widget_array)

In [105]:
# SIGHTS

def get_starfixes (drp_pos : LatLonGeodetic) -> SightCollection :
    ''' Returns a list of used star fixes (SightCollection) '''

    Sight.set_estimated_position (drp_pos)
    retval = []
    for i in range (3):
        if str2bool(numDict["Use"+str(i+1)]):
            retval.append (Sight ( object_name          = numDict         [typeArray[0][0]+str(i+1)],
                                   measured_alt         = numDict         [typeArray[1][0]+str(i+1)],
                                   set_time             = numDict         [typeArray[2][0]+str(i+1)],
                                   index_error_minutes  = float(numDict   [typeArray[3][0]+str(i+1)]),
                                   limb_correction      = int(numDict     [typeArray[4][0]+str(i+1)]),
                                   artificial_horizon   = str2bool(numDict[typeArray[5][0]+str(i+1)]),
                                   observer_height      = float(numDict   [typeArray[6][0]+str(i+1)]),
                                   temperature          = float(numDict   [typeArray[7][0]+str(i+1)]),
                                   dt_dh                = float(numDict   [typeArray[8][0]+str(i+1)]),
                                   pressure             = float(numDict   [typeArray[9][0]+str(i+1)])
                                 ))

    return SightCollection (retval)


In [ ]:
# SIGHT REDUCTION.

the_pos = LatLonGeodetic (float(numDict["DrpLat"]), 
                          float(numDict["DrpLon"])) # Rough DRP position

intersections = None
collection = None
the_map = None
try:
    intersections, _, _, collection =\
            SightCollection.get_intersections_conv (return_geodetic=True,
                                                    estimated_position=the_pos,
                                                    get_starfixes=get_starfixes,
                                                    assume_good_estimated_position=True)
    
    assert intersections is not None
    assert collection is not None
    print (get_representation(intersections,1))
    assert isinstance (intersections, LatLonGeodetic)
    print ("Google Map Coordinate = " + get_google_map_string(intersections,4))

except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve))
    if ve.coll_object is not None:
        if isinstance (ve.coll_object, SightCollection):
            collection = ve.coll_object

if isinstance (intersections, tuple):
    intersections = None

if collection is not None:
    the_map = collection.render_folium (intersections)
the_map